# Продвинутый Python, лекция 10

**Лектор:** Петров Тимур

**Семинаристы:** Петров Тимур, Коган Александра, Романченко Полина

**Spoiler Alert:** в рамках курса нельзя изучить ни одну из тем от и до досконально (к сожалению, на это требуется больше времени, чем даже 3 часа в неделю). Но мы попробуем рассказать столько, сколько возможно :)

Так как мы проходим веб-разработку, то это явно не та вещь, которая делается через colab. Поэтому здесь написан просто код, который можно воспроизвести локально

# Django

## Начала

Как мы уже говорили, если Flask - это буквально сделай все сам (и в этом его плюс и минус), то уже Django - это все сделано за тебя, пользуйся

Давайте смотреть, что сделано и что тогда нам надо делать

По классике вначале просто попробуем установить и создать некоторый проект:

In [ ]:
!pip install django

In [5]:
!django-admin startproject mysite

In [8]:
!cd mysite; ls -la; cd mysite; ls -la

total 16
drwxr-xr-x 3 root root 4096 Nov 13 10:41 .
drwxr-xr-x 1 root root 4096 Nov 13 10:41 ..
-rwxr-xr-x 1 root root  662 Nov 13 10:41 manage.py
drwxr-xr-x 2 root root 4096 Nov 13 10:41 mysite
total 24
drwxr-xr-x 2 root root 4096 Nov 13 10:41 .
drwxr-xr-x 3 root root 4096 Nov 13 10:41 ..
-rw-r--r-- 1 root root  389 Nov 13 10:41 asgi.py
-rw-r--r-- 1 root root    0 Nov 13 10:41 __init__.py
-rw-r--r-- 1 root root 3240 Nov 13 10:41 settings.py
-rw-r--r-- 1 root root  748 Nov 13 10:41 urls.py
-rw-r--r-- 1 root root  389 Nov 13 10:41 wsgi.py


Какие файлы видим?

* manage.py - утилита, позволяющая взаимодействовать с проектом различными способами

* __ init __.py

* asgi.py - точка входа для ASGI-совместимых веб-серверов для обслуживания вашего проекта (асинхронная коммуникация между сервером и приложением)

* settings.py - настройки нашего приложения

* urls.py - маппинг между views и страницами (что где выдавать)

* wsgi.py - Точка входа для WSGI совместимых веб-серверов для работы с проектом

Давайте запустим по дефолту и посмотрим!

Видим совсем базовую страничку, что у нас все заработало

## Время писать что-то свое!

Сделаем простое задание - создать страничку, которая будет отображать текущее время (в UTC). Для этого, как и во Flask, надо написать страничку, которая это будет показывать. Создадим файл views.py и запишем туда:

In [ ]:
from django.http import HttpResponse #внутри django зя ответы можно использовать модуль http
import datetime

def current_datetime(request):
    now = datetime.datetime.now() # Время
    html = "<html><body>It is now %s.</body></html>" % now # страничка
    return HttpResponse(html) # отправляем страничку через HttpResponse (нашу страницу)

Мы создали так называемый view - функция, которая принимает запрос и по нему выдает ответ (по аналогии с route внутри Flask). Но только тут непонятно, а что за страница, которая это все будет показывать

В этом смысле есть отличие. Маппинг наших views и непосредственно страниц осуществляется через файл urls.py

Импортируем наши views и добавим:

In [ ]:
from django.contrib import admin
from django.urls import path
from djangoProject.views import current_datetime # импортируем функцию

urlpatterns = [
    path("admin/", admin.site.urls), #админка сайта (про нее позже)
    path("", current_datetime) #начальная страница
]

Ура, видим то, что надо!

## Динамические ссылки

Смотрим на urls.py, и кажется, что для каждого сайта придется прописывать url... На самом деле нет, потому что Django тоже умеет в динамические ссылки. Для демонстрации напишем функцию, которая будет говорить, сколько у нас будет времени через n часов (ну вдруг кому-то надо)

In [ ]:
def hours_ahead(request, hour):
    dt = datetime.datetime.now() + datetime.timedelta(hours=hour)
    html = "<html><body>In %s hour(s), it will be %s.</body></html>" % (hour, dt)
    return HttpResponse(html)

А теперь смапим с urls.py:

In [ ]:
from django.contrib import admin
from django.urls import path
from djangoProject.views import current_datetime, hours_ahead

urlpatterns = [
    path("admin/", admin.site.urls),
    path("", current_datetime),
    path("<int:hour>/", hours_ahead) #Внутри Django можно осуществлять т.н. захват переменных из URLа
]

А можно писать и просто регулярки через re_path (можно указывать регулярки)

Какие есть динамические захваты по умолчанию?

* str, int - все понятно

* slug - любой набор букв-цифры-подчеркиваний

* path - путь

* uuid - 123e4567-e89b-12d3-a456-426655440000 (канонический вид идентификатора, состоит из 16-ичных числе)

## Шаблоны

Смогли сгенерировать ссылки - супер! А теперь под них надо делать страницы (и как обычно, мы не хотим для каждой писать свой собственный шаблон). Внутри Django есть внутренняя шаблонизация

Основана на двух вещах: встроенная шаблонизация Django + Jinja2 (то есть без определенных деталей same из Flask). Давайте отрендерим самый базовый шаблон

In [ ]:
from django.template import Template

t = Template("My name is {{ name }}.") #строим template и получаем объект типа Template
print(t)

Как в него запихнуть значение переменной? С помощью словаря контекста

In [ ]:
from django.template import Context, Template

t = Template("My name is {{ name }}.")
c = {"name": "Oleg"} # контекст - это по существу словарь с необходимыми данными
t.render(c) # Получаем то, что нужно

Давайте завернем это в view:

In [ ]:
def template_func(request):
    t = Template("My name is {{ name }}.")
    return HttpResponse(t.render({"name": "Oleg"}))  # Получаем то, что нужно

Урааа, получилось! Суть простая: создаем шаблон, запихиваем внутри него необходимые переменные и живем-живем. Теперь давайте попробуем все сделать с помощью не шаблона из string, в из шаблона в отдельном файле. Для этого есть папочка templates. Запихнем все в layout.html

In [ ]:
from django.template.loader import get_template

def template_func(request):
    t = get_template("layout.html")
    c = {"name": "Oleg"}  # контекст - это по существу словарь с необходимыми данными
    return HttpResponse(t.render(c))  # Получаем то, что нужно

Но можно еще проще сделать!

In [ ]:
from django.shortcuts import render

def template_func(request):
    return render(request, "layout.html", context={"name": "Oleg"})  # Скажем по-тупому: отрендери все это)

И еще проще!

In [ ]:
from django.shortcuts import render

def template_func(request):
    name = "Oleg"
    return render(request, "layout.html", locals())  # передай все локальные переменные в качестве словаря со значениями

Чего не хватает для красоты? Правильно, редиректов:



```
<a href="{% url 'index' %}"> Go back </a>
```

Но для этого нужно в urls дать name для ссылки

## Ошибки

Создали шаблоны, urlы, теперь надо бы еще ошибки добавить, верно?

Тут все просто, Django уже постарался, чтобы с этим не приходилось работать. Добавляем в urls вот такие штуки (это наши хэндлеры ошибок)

```

from django.conf.urls import handler400, handler403, handler404, handler500

```

И далее:

```
handler404 = 'djangoProject.views.handler404'
handler500 = 'djangoProject.views.handler500' 
```

И далее во views указываем функции и обрабатываем их. И тогда при любой ошибке будет выскакивать именно нужная страница

Но мы хотим еще и вызывать ошибки, а не только их обрабатывать, ведь так? Давайте делать:

In [ ]:
from django.http import Http404

raise(Http404("Error"))

# SQL

Для того, чтобы двинуться дальше (на семинаре мы поговорим про полную сборку с админкой, БД и так далее), нам необходимо разобраться базово с SQL

Полезный инструмент: https://sql-ex.ru/ (пожалуйста, попрактикуйтесь на нем)

А мы разберем базово этот язык запросов к таблицам (мы говорим про конкретную СУБД, а основы, которые верны везде)

## Структура

Внутри SQL выделяют следущие группы запросов):

* DDL (Data Definition Language) - для создания/изменения/удаления таблиц

* DML (Data Mannipulation Language) - для манипуляции данными

* DCL (Data Control Language) - для доступов к данным

* TCL (Transaction Control Language) - управление транзакциями

Мы поговорим только про DDL и DML (потому что это самое важное)

### DDL

Есть 3 команды:

* CREATE - создай таблицу

```
CREATE TABLE [Name](
    [column_1] <type>,
    [column_2] <type>,
    ...
) - создай таблицу с названием Name с такими колонками
```

* ALTER - измени таблицу

```
ALTER TABLE [Name] ALTER COLUMN [column] <type> - поменяй в таблице колонку, поставив нужный тип
```

* DROP - удали таблицу

```
DROP TABLE [Name]- удали таблицу Name

```

### DML

Структура запроса:

```
SELECT <columns>
FROM <table>
[WHERE]
[GROUP BY]
[HAVING]
[ORDER BY]
[LIMIT] 
```

### Join the Navy!

```
select
    a.<>,
    b.<>
from table_1 as a
[LEFT|RIGHT|INNER] JOIN table_2 as b
ON a.<> = b.<>
```

# Птица дня

![](https://s.mediasalt.ru/images/245/245114/original.jpg)

Это индийский медоуказчик. По его названию в целом можно понять, что он делает. Правильно, громко кричит для того, чтобы приманить людей или медоедов к ульям. приведя к нужному месту, они начинают летать с дерева на дерево в ожидании, и когда кто-то разграбит улей и выведет всех пчел, она питается сотами (вообще удивительно, очень мало кто ест пчелиные соты)

При этом полностью им доверять не следует: они кричат и приводят не только к пчелам, но и к леопардам, к собакам, к логову и так далее (почему - да непонятно). Как вариант - они надеятся потом насладиться червями от убитой туши. Но вообще считается, что это такой вариант коэволюции с человеком, который пользовался их услугами еще 2млн лет назад

По-латински они зовутся максимально просто - Indicator. А еще они как кукушки - кладут яйца к другим птицам